

- How to deal with different columns in generation data for old (`DE_AT_LU` until 2018/09/30) and new bidding zone (`DE_LU` since 2018/10/01)? Old data contains all columns from new data but also additional columns, mostly about `'Actual Consumption'`, and one extra category `'Fossil Coal-derived gas Actual Aggregated'`.
- Which time span to include in general for training data?

## Data-loading playground with `entsoe-py`

### Imports

In [1]:
import pandas as pd
import plotly.express as px

### Settings

In [2]:
api_key = "6e68642c-8403-4caa-af31-bda40b8c67f6" # web token for RESTful API
country_code = "10Y1001A1001A83F" # Germany
BZ_code = "DE_LU" # new bidding zone, valid since 2018/10/01
BZ_code_old = "DE_AT_LU" # old bidding zone, valid until 2018/09/30
time_zone = "Europe/Berlin" # time zone for Germany

### Helper functions

In [3]:
def get_col_diff_intersect(df1, df2):
    """
    Return difference and intersection of columns of two dataframes.
    
    Params
    ------
    df1 : pandas.DataFrame
          first dataframe
    df2 : pandas.DataFrame
          second dataframe
          
    Returns
    -------
    difference in columns of df1 and df2
    intersection of columns of df1 and df2
    
    """
    return df1.columns.difference(df2.columns), df1.columns.intersection(df2.columns)

In [4]:
def get_load_intervals(start_date, 
                       end_date, 
                       time_zone="Europe/Berlin"):
    """
    Get time points for sequential data loading from ENTSO-E transparency platform.
    
    For one request, the time delta for loading data from the platform is limited to one year.
    
    Params
    ------
    start_date : str
                 start date as "yyyymmdd"
    end_date : str
               end date as "yyyymmdd"
    time_zone : str
                time zone as string, e.g. "Europe/Berlin"
    
    Returns
    -------
    pd.Series
    pandas series with timestamps of time points to consider between start and end date
    """
    # Convert start and end dates to timestamps.
    start = pd.Timestamp(start_date, tz=time_zone)
    end = pd.Timestamp(end_date, tz=time_zone)

    # Create series from start and end timestamps.
    start_series = pd.Series(pd.Timestamp(start_date))
    end_series = pd.Series(pd.Timestamp(end_date))
    
    # Create date range from start and end dates and determine year starts within range.
    # Convert data range to series.
    dates = pd.date_range(start=start_date, end=end_date, freq="YS", inclusive="both").to_series()

    # Check whether start date itself is year start.
    # If not, prepend to dates to consider for data loading.
    if not start.is_year_start:
        dates = pd.concat([start_series, dates], ignore_index=True)

    # Check whether end date itself is year start.
    # If not, append to dates to consider for data loading.
    if not end.is_year_start:
        dates = pd.concat([dates, end_series], ignore_index=True)
        
    return dates

In [5]:
def load_data(start_date, 
              end_date, 
              api_key, 
              country_code="10Y1001A1001A83F", 
              time_zone="Europe/Berlin"):
    """
    Load actual load and actual aggregated gemeratopm per production type for requested time interval.
    
    
    Params
    ------
    start_date : str
                 start date as "yyyymmdd"
    end_date : str
               end date as "yyyymmdd"
    api_key : str
              RESTful API web key
    country_code : str
                   code for country, bidding zone, etc.
    time_zone : str
                time zone as string, e.g. "Europe/Berlin"
                
    Returns
    -------
    pd.DataFrame with time points as indices and load + generation per type as columns.
    """
    from entsoe import EntsoePandasClient
    # Initialize client and settings.
    client = EntsoePandasClient(api_key=api_key)
    start = pd.Timestamp(start_date, tz=time_zone)
    end = pd.Timestamp(end_date, tz=time_zone)
    # Query data and save to dataframe.
    df_load = client.query_load(country_code, start=start, end=end)
    print(f"Actual load has shape {df_load.shape}.")
    df_gen = client.query_generation(country_code, start=start, end=end, psr_type=None)
    df_gen.columns = [" ".join(a) for a in df_gen.columns.to_flat_index()]
    print(f"Actual generation per production type has shape {df_gen.shape}.")
    df_final = pd.concat([df_load, df_gen], axis=1) # Concatenate dataframes in columns dimension.
    print(f"Concatenated data frame has shape {df_final.shape}.")
    
    return df_final

In [6]:
def fetch_data(start_date, 
               end_date, 
               api_key, 
               country_code="10Y1001A1001A83F", 
               time_zone="Europe/Berlin"):
    """
    Fetch data from ENTSO-E transparency platform as requested.
    
    Parameters
    ----------
    start_date : str
                 start date as "yyyymmdd"
    end_date : str
               end date as "yyyymmdd"
    api_key : str
              RESTful API web key
    time_zone : str
                time zone as string, e.g. "Europe/Berlin"
    country_code : str
                   code for country, bidding zone, etc.
    
    Returns
    -------
    pd.DataFrame with actual load and generation per type for requested time interval
    """
    # Determine sequence of dates to consider when loading data.
    dates = get_load_intervals(start_date, end_date, time_zone)
    print(f"Consider the following dates:\n{dates}")
    df_list = []
    
    for i, _ in enumerate(dates):

        if i == dates.shape[0] - 1:
            print("Returning final data frame...")
            return pd.concat(df_list, axis=0) # Concatenate dataframes along time axis (index).
            
        try:
            print(f"Trying to load data chunk for time interval [{dates[i]}, {dates[i+1]}]...")
            df_temp = load_data(start_date=dates[i], 
                                end_date=dates[i+1],
                                api_key=api_key,
                                time_zone=time_zone,
                                country_code=country_code)
            print(df_temp.shape)
            df_list.append(df_temp)
            print("Loading successful!")
            
        except Exception as e:
            print(f"Loading failed!", e)
            continue

In [7]:
start_date = "20140101"
end_date = "20230101"
df_test = fetch_data(start_date, end_date, api_key)

Consider the following dates:
2014-01-01   2014-01-01
2015-01-01   2015-01-01
2016-01-01   2016-01-01
2017-01-01   2017-01-01
2018-01-01   2018-01-01
2019-01-01   2019-01-01
2020-01-01   2020-01-01
2021-01-01   2021-01-01
2022-01-01   2022-01-01
2023-01-01   2023-01-01
Freq: AS-JAN, dtype: datetime64[ns]
Trying to load data chunk for time interval [2014-01-01 00:00:00, 2015-01-01 00:00:00]...
Loading failed! 
Trying to load data chunk for time interval [2015-01-01 00:00:00, 2016-01-01 00:00:00]...
Actual load has shape (35040, 1).
Actual generation per production type has shape (35040, 33).
Concatenated data frame has shape (35040, 34).
(35040, 34)
Loading successful!
Trying to load data chunk for time interval [2016-01-01 00:00:00, 2017-01-01 00:00:00]...
Actual load has shape (35136, 1).
Actual generation per production type has shape (35136, 33).
Concatenated data frame has shape (35136, 34).
(35136, 34)
Loading successful!
Trying to load data chunk for time interval [2017-01-01 00:

In [8]:
print(df_test.shape)
print(df_test["Actual Load"].isna().sum())

(279599, 34)
90


### Complete parameter list
https://transparency.entsoe.eu/content/static_content/Static%20content/web%20api/Guide.html#_complete_parameter_list

### Queries returning Pandas Series

`client.query_day_ahead_prices(country_code, start=start,end=end)` <br>
`client.query_net_position(country_code, start=start, end=end, dayahead=True)` <br>
`client.query_crossborder_flows(country_code_from, country_code_to, start, end)` <br>
`client.query_scheduled_exchanges(country_code_from, country_code_to, start, end, dayahead=False)` <br>
`client.query_net_transfer_capacity_dayahead(country_code_from, country_code_to, start, end)` <br>
`client.query_net_transfer_capacity_weekahead(country_code_from, country_code_to, start, end)` <br>
`client.query_net_transfer_capacity_monthahead(country_code_from, country_code_to, start, end)` <br>
`client.query_net_transfer_capacity_yearahead(country_code_from, country_code_to, start, end)` <br>
`client.query_intraday_offered_capacity(country_code_from, country_code_to, start, end,implicit=True)` <br>
`client.query_offered_capacity(country_code_from, country_code_to, start, end, contract_marketagreement_type, implicit=True)` <br>
`client.query_aggregate_water_reservoirs_and_hydro_storage(country_code, start, end)`

### Queries returning Pandas DataFrames

`client.query_load(country_code, start=start,end=end)` <br>
`client.query_load_forecast(country_code, start=start,end=end)` <br>
`client.query_load_and_forecast(country_code, start=start, end=end)` <br>
`client.query_generation_forecast(country_code, start=start,end=end)` <br>
`client.query_wind_and_solar_forecast(country_code, start=start,end=end, psr_type=None)` <br>
`client.query_generation(country_code, start=start,end=end, psr_type=None)` <br>
`client.query_generation_per_plant(country_code, start=start,end=end, psr_type=None)` <br>
`client.query_installed_generation_capacity(country_code, start=start,end=end, psr_type=None)` <br>
`client.query_installed_generation_capacity_per_unit(country_code, start=start,end=end, psr_type=None)` <br>
`client.query_imbalance_prices(country_code, start=start,end=end, psr_type=None)` <br>
`client.query_contracted_reserve_prices(country_code, start, end, type_marketagreement_type, psr_type=None)` <br>
`client.query_contracted_reserve_amount(country_code, start, end, type_marketagreement_type, psr_type=None)` <br>
`client.query_unavailability_of_generation_units(country_code, start=start,end=end, docstatus=None, periodstartupdate=None, periodendupdate=None)` <br>
`client.query_unavailability_of_production_units(country_code, start, end, docstatus=None, periodstartupdate=None, periodendupdate=None)` <br>
`client.query_unavailability_transmission(country_code_from, country_code_to, start, end, docstatus=None, periodstartupdate=None, periodendupdate=None)` <br>
`client.query_withdrawn_unavailability_of_generation_units(country_code, start, end)` <br>
`client.query_import(country_code, start, end)` <br>
`client.query_generation_import(country_code, start, end)` <br>
`client.query_procured_balancing_capacity(country_code, start, end, process_type, type_marketagreement_type=None)`

## Load data from client

In [ ]:
df = pd.DataFrame()
df["load forecast"] = client.query_load_forecast(country_code, start=start,end=end)
df["load"] = client.query_load(country_code, start=start,end=end)
df["load forecast error"] = df["load forecast"] - df["load"]
df["generation forecast"] = client.query_generation_forecast(country_code, start=start,end=end)

In [ ]:
df_gen = client.query_generation(country_code, start=start,end=end, psr_type=None)
df_gen.head()

In [ ]:
df_gen.xs(key="Actual Aggregated", level=1, axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(14,7))
ax.plot(df_gen.xs(key="Actual Aggregated", level=1, axis=1))
ax.legend()

In [ ]:
df["generation"] = df_gen.sum(axis=1)

In [ ]:
df["generation forecast error"] = df["generation forecast"] - df["generation"]

In [ ]:
px.line(df)

## Save to csv file

In [ ]:
df.to_csv('entsoe.csv')